In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
print(tf.__version__)

2.3.0


In [3]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value=w_init(shape=(input_dim, units), dtype="float32"),
            trainable=True,
        )
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(
            initial_value=b_init(shape=(units,), dtype="float32"), trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [4]:
x = tf.ones((2, 2))
linear_layer = Linear(4, 2)
y = linear_layer(x)
print(y)

tf.Tensor(
[[-0.08354362 -0.10411751  0.04516791  0.05773728]
 [-0.08354362 -0.10411751  0.04516791  0.05773728]], shape=(2, 4), dtype=float32)


In [5]:
linear_layer.weights

[<tf.Variable 'Variable:0' shape=(2, 4) dtype=float32, numpy=
 array([[-0.03522181, -0.03715957,  0.02508715, -0.01741817],
        [-0.04832181, -0.06695794,  0.02008076,  0.07515546]],
       dtype=float32)>,
 <tf.Variable 'Variable:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]

还可以使用一种更加快捷的方式为层添加权重：add_weight() 方法：

In [6]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        self.w = self.add_weight(
            shape=(input_dim, units), initializer="random_normal", trainable=True
        )
        self.b = self.add_weight(shape=(units,), initializer="zeros", trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b


x = tf.ones((2, 2))
linear_layer = Linear(4, 2)
y = linear_layer(x)
print(y)

tf.Tensor(
[[-0.04503764 -0.06588107  0.05918322  0.00651141]
 [-0.04503764 -0.06588107  0.05918322  0.00651141]], shape=(2, 4), dtype=float32)


注意  trainable 参数

将权重创建推迟到得知输入的形状之后

在层的 build(self, inputs_shape) 方法中创建层的权重

In [9]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32):
        super(Linear, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [10]:
linear_layer = Linear(4)

In [11]:
linear_layer.weights

[]

In [12]:
linear_layer.build((2,2))

In [13]:
linear_layer.weights

[<tf.Variable 'Variable:0' shape=(2, 4) dtype=float32, numpy=
 array([[ 0.05791345,  0.0387747 ,  0.04984419, -0.01097031],
        [ 0.02050766, -0.02459068,  0.04834839, -0.04136579]],
       dtype=float32)>,
 <tf.Variable 'Variable:0' shape=(4,) dtype=float32, numpy=
 array([-1.1634522e-02,  3.1034455e-05,  6.5388836e-02, -6.9463156e-02],
       dtype=float32)>]

In [8]:
x = tf.ones((2, 2))
linear_layer = Linear(4)
y = linear_layer(x)
print(y)

tf.Tensor(
[[-0.00205199  0.09795983  0.05675608 -0.0306036 ]
 [-0.00205199  0.09795983  0.05675608 -0.0306036 ]], shape=(2, 4), dtype=float32)


层的 __call__() 方法将在首次调用时自动运行构建训练参数

### 层可递归组合

如果将一个层实例分配为另一个层的特性，则外部层将开始跟踪内部层的权重。

我们建议在 __init__() 方法中创建此类子层（由于子层通常具有构建方法，它们将与外部层同时构建）。

In [10]:
class MLPBlock(keras.layers.Layer):
    def __init__(self):
        super(MLPBlock, self).__init__()
        self.linear_1 = Linear(32)
        self.linear_2 = Linear(32)
        self.linear_3 = Linear(1)

    def call(self, inputs):
        x = self.linear_1(inputs)
        x = tf.nn.relu(x)
        x = self.linear_2(x)
        x = tf.nn.relu(x)
        return self.linear_3(x)

In [11]:
mlp = MLPBlock()

通常使用 Layer 类来定义内部计算块，并使用 Model 类来定义外部模型，即您将训练的对象。

Model 类具有与 Layer 相同的 API，但有如下区别：

它会公开内置训练、评估和预测循环（model.fit()、model.evaluate()、model.predict()）。

它会通过 model.layers 属性公开其内部层的列表。

它会公开保存和序列化 API（save()、save_weights()…）

因此，如果您想知道“我应该用 Layer 类还是 Model 类？”，

请问自己：我是否需要在它上面调用 fit()？

我是否需要在它上面调用 save()？

如果是，则使用 Model。如果不是（要么因为您的类只是更大系统中的一个块，要么因为您正在自己编写训练和保存代码），则使用 Layer。